In [1]:
import pandas as pd

In [2]:
data = pd.read_json("data/feed_news.json")

ValueError: Expected object or value

In [3]:
data

,category,body,subtitle,title,url,spider,date_edit,source,date_retrieved,keywords,date_published,author
0,Economia,A renda média mensal por pessoa dos brasileiro...,Diferença de renda per capita entre municípios...,Diferença de renda per capita entre municípios...,http://g1.globo.com/economia/noticia/2013/07/d...,globo_g1,2013-07-29T19:42:00,"Do G1, em São Paulo",2013-07-30T01:06:53.462909,"notícias, notícia, Economia",2013-07-29T19:18:00,NaN
1,Economia,Segunda maior empresa de shopping centers dos ...,GGP vende fatias na Aliansce para Canada Pensi...,GGP vende fatias na Aliansce para Canada Pensi...,http://g1.globo.com/economia/noticia/2013/07/g...,globo_g1,2013-07-29T19:30:00,NaN,2013-07-30T01:06:53.535095,"notícias, notícia, Economia",2013-07-29T19:30:00,Valor OnLine
2,G1 - Agronegócios,"Brasília, 29 - A Companhia Nacional de Abastec...",Governo retoma leilões de estoques de arroz no...,Governo retoma leilões de estoques de arroz no...,http://g1.globo.com/economia/agronegocios/noti...,globo_g1,2013-07-29T19:50:00,NaN,2013-07-30T01:06:53.576616,"notícias, notícia, Agronegócios",2013-07-29T19:50:00,Agencia Estado
3,Economia,"NOVA YORK, Nova Iorque, 29 Jul 2013 (AFP) - Wa...","Wall Street fecha em queda: Dow Jones -0,24%, ...","Wall Street fecha em queda: Dow Jones -0,24%, ...",http://g1.globo.com/economia/noticia/2013/07/w...,globo_g1,2013-07-29T19:25:00,NaN,2013-07-30T01:06:53.587348,"notícias, notícia, Economia",2013-07-29T19:25:00,France Presse
4,Economia,"BRASÍLIA, 29 Jul (Reuters) - O ministro dos T...",Taxa de retorno de concessões de ferrovias ser...,Taxa de retorno de concessões de ferrovias ser...,http://g1.globo.com/economia/noticia/2013/07/t...,globo_g1,2013-07-29T19:50:00,NaN,2013-07-30T01:06:53.620488,"notícias, notícia, Economia",2013-07-29T19:50:00,Reuters
...,...,...,...,...,...,...,...,...,...,...,...,...
3877,Auto Esporte,Basta encarar por dez segundos o novo RAV4 e d...,Primeiras impressões: Toyota RAV4 2.0 4x2 e 2....,Primeiras impressões: Toyota RAV4 2.0 4x2 e 2....,http://g1.globo.com/carros/noticia/2013/05/pri...,globo_g1,2013-05-08T08:15:00,De Itatiba (SP),2013-07-30T01:19:25.936485,"notícias, notícia, Auto Esporte, Últimas notícias",2013-05-08T08:15:00,Rodrigo Mora
3878,Auto Esporte,O redesenhado Chevrolet Camaro SS vai estrear ...,Novo Chevrolet Camaro SS será revelado no Salã...,Novo Chevrolet Camaro SS será revelado no Salã...,http://g1.globo.com/carros/noticia/2013/03/nov...,globo_g1,2013-03-21T10:06:00,"Do G1, em São Paulo",2013-07-30T01:19:25.995659,"notícias, notícia, Auto Esporte, Últimas notícias",2013-03-21T09:52:00,NaN
3879,G1 - Acre,O advogado da Telexfree Horst Fouchs está no ...,Advogado da Telexfree diz que está tomando 'me...,Advogado da Telexfree diz que está tomando 'me...,http://g1.globo.com/ac/acre/noticia/2013/06/ad...,globo_g1,2013-07-08T13:36:00,Do G1 AC,2013-07-30T01:19:26.036687,"notícias, notícia, Acre",2013-06-20T11:24:00,Yuri Marcel
3880,G1 - Tecnologia e Games,As operadoras de telecomunicações Claro e TIM ...,Saem preços do iPad mini no Brasil,Saem preços do iPad mini no Brasil,http://g1.globo.com/tecnologia/noticia/2013/06...,globo_g1,2013-06-25T08:34:00,"Do G1, em São Paulo",2013-07-30T01:19:28.027228,"notícias, notícia, Tecnologia e Games, Últimas...",2013-06-24T18:20:00,NaN


In [ ]:
!pip install -U spacy 
!pip install tf-estimator-nightly # tensorflow requer tf-estimator-nightly.
!python -m spacy download pt_core_news_lg 

ERROR: Invalid requirement: '#'


In [ ]:
import spacy
nlp = spacy.load("pt_core_news_lg")

In [ ]:
from tqdm.auto import tqdm 
tqdm.pandas()

In [ ]:
data['docs'] = data.body.progress_map(nlp)

  0%|          | 0/3882 [00:00<?, ?it/s]

In [ ]:
#
data.to_pickle("documents.pickle")

In [ ]:
companies = []
for doc in data.docs:
    orgs = []
    for ent in doc.ents:
        if ent.label_ != "ORG":
            continue
        orgs.append(ent.text)
    orgs = list(set(orgs))
    if len(orgs) > 1:
        companies.append(orgs)


In [ ]:
companies[0:2]

[['Sena-MA',
  'RR',
  'Programa das Nações Unidas para o Desenvolvimento',
  'Pnud'],
 ['General Growth Properties',
  'Aliansce',
  'Aliansce para a Canada Pension Plan Investment Board',
  'CPPIB',
  'GGP']]

In [ ]:
import itertools
edges = []

for nodes in companies:
    edges.extend([[n1, n2] for n1, n2 in itertools.combinations(nodes, 2)])

edges = pd.DataFrame(edges, columns=['source', 'target'])
edges

,source,target
0,Sena-MA,RR
1,Sena-MA,Programa das Nações Unidas para o Desenvolvimento
2,Sena-MA,Pnud
3,RR,Programa das Nações Unidas para o Desenvolvimento
4,RR,Pnud
...,...,...
78873,Samsung,Apple
78874,Samsung,IDC
78875,Amazon,Apple
78876,Amazon,IDC


In [ ]:
edges_weighted = edges.groupby(edges.columns.tolist(), as_index=False).size().sort_values(by="size", ascending=False)

In [ ]:
edges_weighted.columns = ['source', 
                          'target',
                          'weight']
edges_weighted                      

,source,target,weight
2656,Anatel,Agência Nacional de Telecomunicações,104
5802,Banco Central,BC,103
25211,Google,Apple,101
43383,Samsung,Apple,95
24521,Games,Tecnologia,94
...,...,...,...
18882,"Estatística, Psicologia e Relações Internacion...",Mecânica,1
18883,"Estatística, Psicologia e Relações Internacion...",Secretariado Executivo,1
18884,"Estatística, Psicologia e Relações Internacion...",Técnico em Eletrotécnica,1
18885,"Estatística, Psicologia e Relações Internacion...",Técnico em Telecomunicações,1


In [ ]:
import networkx as nx 

In [ ]:
graph = nx.from_pandas_edgelist(edges_weighted)

In [ ]:
graph

In [ ]:
nx.write_graphml(graph, 'companies.graphml')